In [1]:
#Dependencies and Set Up
import requests
import json
import pandas as pd
import reverse_geocoder as rg
import os
import csv

from pprint import pprint
from datetime import datetime as dt

In [2]:
#Build a parameter and url in order to request data from API
paramss = {"format": "geojson", 
           "starttime": "2010-01-01", 
           "endtime": "2020-12-31"}

url = r"https://earthquake.usgs.gov/fdsnws/event/1/query?"

#Create list for the different alert levels, and an empty data list to append data in it
alert_levels=['green','yellow','orange','red']
data_list = []

def earthquake(level):
    
    #Function to get the json data from earthquake.usgs.gov
    paramss['alertlevel'] = level
    print(paramss)
    try:
        response = requests.get(url, params = paramss)
        print(response.status_code)
        if response.status_code == 200:
            data = json.loads(response.text)
            #pprint(data)

    except requests.exceptions.RequestException as e:
        print(e)
    finally:
        return data

In [8]:
def getJsonData():
    for level in alert_levels:
        data = earthquake(level)
        if data:
            data_list.append(data)
    
    #save to .json file
    with open('data.json', 'w') as outfile:
        json.dump(data_list, outfile)

getJsonData()  
print('JSON file created.')

{'format': 'geojson', 'starttime': '2010-01-01', 'endtime': '2020-12-31', 'alertlevel': 'green'}
200
{'format': 'geojson', 'starttime': '2010-01-01', 'endtime': '2020-12-31', 'alertlevel': 'yellow'}
200
{'format': 'geojson', 'starttime': '2010-01-01', 'endtime': '2020-12-31', 'alertlevel': 'orange'}
200
{'format': 'geojson', 'starttime': '2010-01-01', 'endtime': '2020-12-31', 'alertlevel': 'red'}
200
JSON file created.


In [9]:
#Opening JSON file
f = open('data.json',)
 
#Returns JSON object as a dictionary
data = json.load(f)

#Create a list to append values and data required for analysis
earthquake_subdata= []

In [11]:
# Iterating through the json list

for feature in data[0]['features']:
    #pprint(feature)
    dEarthquake = {}
    #print(feature['id'])
    dEarthquake['id']= feature['id']
    dEarthquake['lat']= feature['geometry']['coordinates'][0]
    dEarthquake['lng']= feature['geometry']['coordinates'][1]
    dEarthquake['latlngdepth']=feature['geometry']['coordinates']
    dEarthquake['alert']= feature['properties']['alert']
    dEarthquake['place']= feature['properties']['place']
    dEarthquake['date']= dt.fromtimestamp(feature['properties']['time']/1000)
    dEarthquake['magnitude']= feature['properties']['mag']
    dEarthquake['tsunami']= feature['properties']['tsunami']
    dEarthquake['type']= feature['properties']['type']
    
    earthquake_subdata.append(dEarthquake)
    #pprint(i)
 
# Closing file
f.close()

#Get the json information that we need and convert it into a dataframe
earthquake_data= pd.DataFrame(earthquake_subdata)

#Create a csv file with all our required data
output_folder = "../output_data/"
earthquake_data.to_csv(output_folder + "EarthquakeData.csv", encoding="utf-8", index = True)